In [1]:
# specify id
y_id = 1

In [2]:
# import module
import sys
sys.path.insert(0, '../')
from utils.training_utils import *
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, make_scorer
from xgboost import XGBRegressor

In [3]:
# specify parameters
params = {
    'booster': ['gbtree', 'dart'],
    'n_estimators': [50, 100, 500],
    'max_depth': [3, 6],
    'subsample': [0.25, 0.5, 0.75],
    'learning_rate': [0.1, 0.05, 0.01], 
    'tree_method': ['hist', 'auto']
}
params_test = {
    'booster': ['dart'],
    'n_estimators': [50],
    'max_depth': [3],
    'subsample': [0.5],
    'learning_rate': [0.05]
}

In [19]:
# load datas
test_x, train_x, train_y = load_data(y_id)
print(test_x.shape, train_x.shape, train_y.shape)

(2500, 10000) (47500, 10000) (47500,)


In [20]:
# pick only important data
idx = []
with open('../29/adaboost' + str(y_id) + '_feature.csv', 'r') as f:
    i = 0
    for lines in f:
        importance = float(lines.replace('\n', '').split(',')[y_id])
        if(np.abs(importance) > 1e-9):
            idx.append(i)
        i += 1
train_x = train_x[:, idx]
test_x = test_x[:, idx]
print(train_x.shape)

(47500, 87)


In [6]:
# create GridSearchCV
model = GridSearchCV(estimator=XGBRegressor(verbosity=2, n_jobs=8), 
                     param_grid=params_test, 
                     scoring=make_scorer(mean_absolute_error),
                     cv=3,
                     verbose=20,
                     n_jobs=4,
                     return_train_score=True)

In [7]:
# train
model.fit(train_x, train_y)

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 tasks      | elapsed:    4.6s
[Parallel(n_jobs=4)]: Done   3 out of   3 | elapsed:    4.8s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   3 out of   3 | elapsed:    4.8s finished
/home/student/07/b07902047/python-packages/xgboost/lib/python3.7/site-packages/xgboost/core.py:613: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn("Use subset (sliced data) of np.ndarray is not recommended " +


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, importance_type='gain',
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, n_jobs=8,
       nthread=None, objective='reg:linear', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=1, verbosity=2),
       fit_params=None, iid='warn', n_jobs=4,
       param_grid={'booster': ['dart'], 'n_estimators': [50], 'max_depth': [3], 'subsample': [0.5], 'learning_rate': [0.05]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=make_scorer(mean_absolute_error), verbose=20)

In [25]:
# write files
write_prediction('train_y0_1.txt', 'w', model.predict(test_x).reshape((2500, 1)).astype('str'))
write_prediction('test_y0_1.txt', 'w', model.predict(test_x).reshape((2500, 1)).astype('str'))

28.741974006299635
